## Tratando dados

In [177]:
import pandas as pd

In [178]:
data = pd.read_csv('star_classification.csv')
data.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [179]:
y = data['class']
x = data.drop('class', axis=1)
x.drop('obj_ID', axis=1, inplace=True)


In [180]:
data.shape

(100000, 18)

In [181]:
data.isnull().sum() / data.shape[0] * 100 # em porecentagem

obj_ID         0.0
alpha          0.0
delta          0.0
u              0.0
g              0.0
r              0.0
i              0.0
z              0.0
run_ID         0.0
rerun_ID       0.0
cam_col        0.0
field_ID       0.0
spec_obj_ID    0.0
class          0.0
redshift       0.0
plate          0.0
MJD            0.0
fiber_ID       0.0
dtype: float64

In [182]:
y.value_counts()

GALAXY    59445
STAR      21594
QSO       18961
Name: class, dtype: int64

## Dados modelo

In [183]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [184]:
scaler = StandardScaler()
x_normalizado = scaler.fit_transform(x)

In [185]:
x_normalizado.shape

(100000, 16)

In [186]:
import sklearn.utils
x_shuffle, y_shuffle = sklearn.utils.shuffle(x_normalizado, y)

In [187]:
# convertendo y em valor númerico
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
y_encoder = label.fit_transform(y_shuffle)

In [188]:
from keras.utils import np_utils
y_one_hot_encoder = np_utils.to_categorical(y_encoder)

## Dividindo dados do modelo

In [189]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x_shuffle, y_one_hot_encoder, test_size=0.3, random_state=42)

In [190]:
lerning_rate = 0.001
epochs = 500
batch_size = 50

In [191]:
n_entrada = 16
n_camada_1 = 10
n_classes = 3

In [192]:
x = tf.placeholder(tf.float32, [None, n_entrada])
y = tf.placeholder(tf.float32, [None, n_classes])

In [193]:
# Pesos da camada 1
w1= tf.Variable(tf.random_normal([n_entrada, n_camada_1], stddev=0.05))
# Bias da camada 1
b1 = tf.Variable(tf.zeros([n_camada_1]))
# Camada 1
layer_1 = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))

# Dropout da camada 1
drop1 = tf.nn.dropout(layer_1, rate=0.2)


# pesos da camada de Saida (output)
w_out = tf.Variable(tf.random_normal([n_camada_1, n_classes], stddev=0.05))
# Bias da camada de saída (output)
bias_out = tf.Variable(tf.zeros([n_classes]))
# Camada de saída (output)
saida = tf.matmul(drop1, w_out) + bias_out


In [194]:
# Função de custo
custo = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=saida, labels=y))
# Otimizador
otimizador = tf.train.AdamOptimizer(learning_rate=lerning_rate).minimize(custo)


In [195]:
# Testando modelo

predicoes = tf.equal(tf.argmax(saida, 1), tf.argmax(y, 1))
acuracia = tf.reduce_mean(tf.cast(predicoes, tf.float32))

In [196]:
tamanho_treino = len(x_treino)
tamanho_treino

70000

In [197]:
total_batches = tamanho_treino / batch_size
total_batches

1400.0

In [198]:
tamanho_treino = x_treino.shape[0]  # Assumindo que a dimensão 0 representa o número de amostras
total_batches = tamanho_treino / batch_size

init = tf.global_variables_initializer()

# Sessão
with tf.Session() as sess:

  sess.run(init)  
  for epoch in range(epochs):
    custo_medio = 0.0

    for i in range(0, tamanho_treino, batch_size):
        batch_x = x_treino[i:i+batch_size]
        batch_y = y_treino[i:i+batch_size]

        sess.run(otimizador, feed_dict={x: batch_x, y: batch_y})

        custo_medio += sess.run(custo, feed_dict={x: batch_x, y: batch_y}) / total_batches

    acuracia_teste = sess.run(acuracia, feed_dict={x: x_teste, y: y_teste})

    print("Epoch", "{},".format(epoch + 1), "Custo médio treino = ", "{:.3f}".format(custo_medio))
    print("Acuracia teste = ", "{:.3f}".format(acuracia_teste))

Epoch 1, Custo médio treino =  0.629
Acuracia teste =  0.854
Epoch 2, Custo médio treino =  0.339
Acuracia teste =  0.903
Epoch 3, Custo médio treino =  0.281
Acuracia teste =  0.918
Epoch 4, Custo médio treino =  0.252
Acuracia teste =  0.926
Epoch 5, Custo médio treino =  0.234
Acuracia teste =  0.928
Epoch 6, Custo médio treino =  0.221
Acuracia teste =  0.934
Epoch 7, Custo médio treino =  0.215
Acuracia teste =  0.936
Epoch 8, Custo médio treino =  0.205
Acuracia teste =  0.938
Epoch 9, Custo médio treino =  0.197
Acuracia teste =  0.941
Epoch 10, Custo médio treino =  0.198
Acuracia teste =  0.942
Epoch 11, Custo médio treino =  0.195
Acuracia teste =  0.941
Epoch 12, Custo médio treino =  0.189
Acuracia teste =  0.943
Epoch 13, Custo médio treino =  0.188
Acuracia teste =  0.943
Epoch 14, Custo médio treino =  0.187
Acuracia teste =  0.945
Epoch 15, Custo médio treino =  0.184
Acuracia teste =  0.944
Epoch 16, Custo médio treino =  0.184
Acuracia teste =  0.945
Epoch 17, Custo m